In [1]:
from tensorflow.keras import backend as K
K.clear_session()

In [2]:
import os
os.chdir("/media/seconddrive/mta_stationing_problem")
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import sys
import datetime as dt
import importlib
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark import SparkConf
import numpy as np
import pickle
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate, GlobalAveragePooling1D
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
import IPython
from copy import deepcopy
from tqdm import trange, tqdm

from src import tf_utils, config, data_utils, models, linklevel_utils

mpl.rcParams['figure.facecolor'] = 'white'

import warnings

import pandas as pd
import swifter
pd.set_option('display.max_columns', None)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.get_logger().setLevel('INFO')

In [4]:
importlib.reload(tf_utils)
importlib.reload(models)

<module 'src.models' from '/media/seconddrive/mta_stationing_problem/src/models.py'>

In [5]:
import pyspark
print(pyspark.__version__)

3.3.0


In [6]:
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '40g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .getOrCreate()

22/08/18 07:12:56 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/08/18 07:12:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/18 07:12:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/18 07:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/18 07:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/18 07:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
f = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)

In [8]:
todelete = apcdata.filter('(load < 0) OR (load IS NULL)').select('transit_date','trip_id','overload_id').distinct()
todelete=todelete.withColumn('marker',F.lit(1))

#joining and whereever the records are not found in sync error table the marker will be null
apcdataafternegdelete=apcdata.join(todelete,on=['trip_id','transit_date','overload_id'],how='left').filter('marker is null').drop('marker')

In [9]:
apcdataafternegdelete = apcdataafternegdelete.sort(['trip_id', 'overload_id'])

In [10]:
get_columns = ['trip_id', 'transit_date', 'arrival_time', 
               'block_abbr', 'stop_sequence', 'stop_id_original',
               'offs', 
               'darksky_temperature', 
               'darksky_humidity', 
               'darksky_precipitation_probability', 
               'route_direction_name', 'route_id',
               'dayofweek',  'year', 'month', 'hour',
               'sched_hdwy']
get_str = ", ".join([c for c in get_columns])

apcdataafternegdelete.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
"""
print(query)

apcdataafternegdelete = spark.sql(query)

22/08/18 07:13:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.

SELECT trip_id, transit_date, arrival_time, block_abbr, stop_sequence, stop_id_original, offs, darksky_temperature, darksky_humidity, darksky_precipitation_probability, route_direction_name, route_id, dayofweek, year, month, hour, sched_hdwy
FROM apc



In [11]:
df = apcdataafternegdelete.toPandas()
print(df.shape)
old_shape = df.shape[0]
df.head(1)

(16683167, 17)


,trip_id,transit_date,arrival_time,block_abbr,stop_sequence,stop_id_original,offs,darksky_temperature,darksky_humidity,darksky_precipitation_probability,route_direction_name,route_id,dayofweek,year,month,hour,sched_hdwy
0,193637,2020-02-21,2020-02-21 23:38:48,1403,39,MEALYTNN,0.0,29.067,0.66,0.0,FROM DOWNTOWN,14,6,2020,2,23.0,3480.0


In [12]:
df = df[df.arrival_time.notna()]
df = df[df.sched_hdwy.notna()]
df = df[df.darksky_temperature.notna()]

df['route_id_dir'] = df["route_id"].astype("str") + "_" + df["route_direction_name"]
df['day'] = df["arrival_time"].dt.day
df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

# Adding extra features
# Holidays
fp = os.path.join('data', 'others', 'US Holiday Dates (2004-2021).csv')
holidays_df = pd.read_csv(fp)
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
holidays_df['is_holiday'] = True
df = df.merge(holidays_df[['Date', 'is_holiday']], left_on='transit_date', right_on='Date', how='left')
df['is_holiday'] = df['is_holiday'].fillna(False)
df = df.drop(columns=['Date'])

# Traffic
# Causes 3M data points to be lost
fp = os.path.join('data', 'traffic', 'triplevel_speed.pickle')
speed_df = pd.read_pickle(fp)
speed_df = speed_df.rename({'route_id_direction':'route_id_dir'}, axis=1)
speed_df = speed_df[['transit_date', 'trip_id', 'route_id_dir', 'traffic_speed']]
df = df.merge(speed_df, how='left', 
                left_on =['transit_date', 'trip_id', 'route_id_dir'], 
                right_on=['transit_date', 'trip_id', 'route_id_dir'])
# df = df[~df['traffic_speed'].isna()]
df['traffic_speed'].bfill(inplace=True)

In [13]:
sorted_df = []
for ba in tqdm(df.block_abbr.unique()):
    ba_df = df[df['block_abbr'] == ba]
    end_stop = ba_df.stop_sequence.max()
    # Same result as creating a fixed_arrival_time (but faster)
    ba_df = ba_df[ba_df.stop_sequence != end_stop].reset_index(drop=True)
    sorted_df.append(ba_df)
        
overall_df = pd.concat(sorted_df)
drop_cols = ['route_direction_name', 'route_id', 'trip_id']
drop_cols = [col for col in drop_cols if col in overall_df.columns]
overall_df = overall_df.drop(drop_cols, axis=1)

# overall_df = overall_df.rename({"fixed_arrival_time": "arrival_time"}, axis=1)

100%|██████████| 227/227 [00:06<00:00, 36.84it/s]


In [14]:
TIMEWINDOW = 15
overall_df['minute'] = overall_df['arrival_time'].dt.minute
overall_df['minuteByWindow'] = overall_df['minute'] // TIMEWINDOW
overall_df['temp'] = overall_df['minuteByWindow'] + (overall_df['hour'] * 60 / TIMEWINDOW)
overall_df['time_window'] = np.floor(overall_df['temp']).astype('int')
overall_df = overall_df.drop(columns=['minute', 'minuteByWindow', 'temp'])

In [15]:
print(overall_df.shape)
overall_df.head(1)

(14371083, 19)


,transit_date,arrival_time,block_abbr,stop_sequence,stop_id_original,offs,darksky_temperature,darksky_humidity,darksky_precipitation_probability,dayofweek,year,month,hour,sched_hdwy,route_id_dir,day,is_holiday,traffic_speed,time_window
0,2020-01-01,2020-01-01 10:01:17,300,1,MCC5_5,0.0,40.15,0.616,0.0,4,2020,1,10.0,4800.0,3_FROM DOWNTOWN,1,True,18.639604,40


In [16]:
# Group by time windows and get the maximum of the aggregate load/class/sched
# Get mean of temperature (mostly going to be equal)
# TODO: Double check this! 
overall_df = overall_df.groupby(['transit_date', 
                                 'route_id_dir', 
                                 'stop_id_original',
                                 'time_window']).agg({"block_abbr":"first",
                                                      "arrival_time":"first",
                                                      "year":"first", 
                                                      "month":"first",
                                                      "day": "first",
                                                      "hour":"first",
                                                      "is_holiday": "first",
                                                      "dayofweek":"first",
                                                      "stop_sequence":"first",
                                                      "darksky_temperature":"mean", 
                                                      "darksky_humidity":"mean",
                                                      "darksky_precipitation_probability": "mean",
                                                      "traffic_speed":"mean",
                                                      "sched_hdwy": "max",
                                                      "offs": "sum" })
overall_df = overall_df.reset_index(level=[0,1,2,3])
overall_df = overall_df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

In [17]:
drop_cols = ['arrival_time', 'block_abbr']
drop_cols = [col for col in drop_cols if col in overall_df.columns]
overall_df = overall_df.drop(drop_cols, axis=1)
overall_df = overall_df[overall_df.offs <= config.TARGET_MAX]

In [18]:
## Hyperparameters
past = 5 # Past stops observed
future = 1 # Future stops predicted
offset = 0

learning_rate = 1e-4
batch_size = 256
epochs = 200

feature_label = config.TARGET_COLUMN_CLASSIFICATION
patience = 10

hyperparams_dict = {'past': past,
                    'future': future,
                    'offset': offset,
                    'learning_rate': learning_rate,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'patience': patience}
hyperparams_dict

{'past': 5,
 'future': 1,
 'offset': 0,
 'learning_rate': 0.0001,
 'batch_size': 256,
 'epochs': 200,
 'patience': 10}

In [19]:
# target = config.TARGET_COLUMN_CLASSIFICATION
target = 'offs'

num_columns = ['darksky_temperature', 'darksky_humidity', 'darksky_precipitation_probability', 'sched_hdwy', 'traffic_speed', target]
cat_columns = ['month', 'hour', 'day', 'stop_sequence', 'stop_id_original', 'year', 'time_window']
ohe_columns = ['dayofweek', 'route_id_dir', 'is_holiday']

columns = num_columns + cat_columns + ohe_columns
print(f"Numerical columns: {num_columns}")
print(f"Categorical columns: {cat_columns}")
print(f"One Hot Encode columns: {ohe_columns}")

Numerical columns: ['darksky_temperature', 'darksky_humidity', 'darksky_precipitation_probability', 'sched_hdwy', 'traffic_speed', 'offs']
Categorical columns: ['month', 'hour', 'day', 'stop_sequence', 'stop_id_original', 'year', 'time_window']
One Hot Encode columns: ['dayofweek', 'route_id_dir', 'is_holiday']


In [20]:
overall_df.head(1)

,transit_date,route_id_dir,stop_id_original,time_window,year,month,day,hour,is_holiday,dayofweek,stop_sequence,darksky_temperature,darksky_humidity,darksky_precipitation_probability,traffic_speed,sched_hdwy,offs
0,2020-01-01,3_FROM DOWNTOWN,MCC5_5,40,2020,1,1,10.0,True,4,1,40.15,0.616,0.0,18.639604,4800.0,0.0


In [21]:
train_dates = ('2020-01-01', '2021-06-30')
val_dates =   ('2021-06-30', '2021-10-31')
test_dates =  ('2021-10-31', '2022-04-06')

ohe_encoder, label_encoder, num_scaler, train_df, val_df, test_df = linklevel_utils.prepare_linklevel(overall_df, 
                                                                                                      train_dates=train_dates, 
                                                                                                      val_dates=val_dates, 
                                                                                                      test_dates=test_dates,
                                                                                                      cat_columns=cat_columns,
                                                                                                      num_columns=num_columns,
                                                                                                      ohe_columns=ohe_columns,
                                                                                                      feature_label='offs',
                                                                                                      time_feature_used='transit_date',
                                                                                                      scaler='minmax')

drop_cols = ['transit_date', 'load', 'arrival_time']
drop_cols = [col for col in drop_cols if col in train_df.columns]
train_df = train_df.drop(drop_cols, axis=1)
val_df = val_df.drop(drop_cols, axis=1)
test_df = test_df.drop(drop_cols, axis=1)

arrange_cols = [target] + [col for col in train_df.columns if col != target]
train_df = train_df[arrange_cols]
val_df = val_df[arrange_cols]
test_df = test_df[arrange_cols]

Train df:  (7751067, 97)
Val df:  (2711795, 97)
Test df:  (3564952, 97)
Columns to drop: ['transit_date', 'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6', 'dayofweek_7', 'route_id_dir_14_FROM DOWNTOWN', 'route_id_dir_14_TO DOWNTOWN', 'route_id_dir_17_FROM DOWNTOWN', 'route_id_dir_17_TO DOWNTOWN', 'route_id_dir_18_FROM DOWNTOWN', 'route_id_dir_18_TO DOWNTOWN', 'route_id_dir_19_FROM DOWNTOWN', 'route_id_dir_19_TO DOWNTOWN', 'route_id_dir_21_NORTHBOUND', 'route_id_dir_21_SOUTHBOUND', 'route_id_dir_22_FROM DOWNTOWN', 'route_id_dir_22_TO DOWNTOWN', 'route_id_dir_23_FROM DOWNTOWN', 'route_id_dir_23_TO DOWNTOWN', 'route_id_dir_24_FROM DOWNTOWN', 'route_id_dir_24_TO DOWNTOWN', 'route_id_dir_25_NORTHBOUND', 'route_id_dir_25_SOUTHBOUND', 'route_id_dir_28_FROM DOWNTOWN', 'route_id_dir_28_TO DOWNTOWN', 'route_id_dir_29_FROM DOWNTOWN', 'route_id_dir_29_TO DOWNTOWN', 'route_id_dir_34_FROM DOWNTOWN', 'route_id_dir_34_TO DOWNTOWN', 'route_id_dir_35_FROM DOWNTOW

In [22]:
## Saving encoders, scalers and column arrangement
fp = os.path.join('models', 'ons_offs', 'LL_OHE_encoder.joblib')
joblib.dump(ohe_encoder, fp)
fp = os.path.join('models', 'ons_offs', 'LL_Label_encoders.joblib')
joblib.dump(label_encoder, fp)
fp = os.path.join('models', 'ons_offs', 'LL_Num_scaler.joblib')
joblib.dump(num_scaler, fp)
fp = os.path.join('models', 'ons_offs', 'LL_X_columns.joblib')
joblib.dump(train_df.columns, fp)

['models/ons_offs/LL_X_columns.joblib']

In [23]:
# Can add shuffle in the future
@tf.autograph.experimental.do_not_convert
def timeseries_dataset_from_dataset(df, feature_slice, label_slice, input_sequence_length, output_sequence_length, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(df.values)
    ds = dataset.window(input_sequence_length + output_sequence_length, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x).batch(input_sequence_length + output_sequence_length)
     
    def split_feature_label(x):
        return x[:input_sequence_length:, feature_slice], x[input_sequence_length:,label_slice]
     
    ds = ds.map(split_feature_label)
     
    return ds.batch(batch_size)


In [24]:
label_index = train_df.columns.tolist().index(target)
print(label_index)

0


In [25]:
label_slice = slice(label_index, label_index + 1, None) # which column the label/labels are
feature_slice = slice(None, None, None) # Which feature columns are included, by default includes all (even label)
input_sequence_length = past # number of past information to look at
output_sequence_length = future # number of time steps to predict

dataset_train = timeseries_dataset_from_dataset(train_df, 
                                                feature_slice=feature_slice,
                                                label_slice=label_slice,
                                                input_sequence_length=input_sequence_length, 
                                                output_sequence_length=output_sequence_length, 
                                                batch_size=batch_size)

dataset_val = timeseries_dataset_from_dataset(val_df, 
                                              feature_slice=feature_slice,
                                              label_slice=label_slice,
                                              input_sequence_length=input_sequence_length, 
                                              output_sequence_length=output_sequence_length, 
                                              batch_size=batch_size)

dataset_test = timeseries_dataset_from_dataset(test_df,
                                               feature_slice=feature_slice,
                                               label_slice=label_slice,
                                               input_sequence_length=input_sequence_length, 
                                               output_sequence_length=output_sequence_length, 
                                               batch_size=batch_size)
for batch in dataset_train.take(1):
    (x, y) = batch
    display(pd.DataFrame(x[100], columns=train_df.columns))
    print(x[100].shape)
    display(pd.DataFrame(y[100], columns=[target]))

2022-08-18 02:17:23.785812: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-08-18 02:17:23.785857: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: scope-vanderbilt
2022-08-18 02:17:23.786130: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: scope-vanderbilt
2022-08-18 02:17:23.786457: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-08-18 02:17:23.786483: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.48.7
2022-08-18 02:17:23.786490: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 515.48.7 does not match DSO version 515.65.1 -- cannot find working devices in this configuration
2022-08-18 02:17:23.787401: I tensorflow/core/platform/cpu_feature_guard.cc:151]

,offs,stop_id_original,time_window,year,month,day,hour,stop_sequence,darksky_temperature,darksky_humidity,darksky_precipitation_probability,traffic_speed,sched_hdwy,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,dayofweek_7,route_id_dir_14_FROM DOWNTOWN,route_id_dir_14_TO DOWNTOWN,route_id_dir_17_FROM DOWNTOWN,route_id_dir_17_TO DOWNTOWN,route_id_dir_18_FROM DOWNTOWN,route_id_dir_18_TO DOWNTOWN,route_id_dir_19_FROM DOWNTOWN,route_id_dir_19_TO DOWNTOWN,route_id_dir_21_NORTHBOUND,route_id_dir_21_SOUTHBOUND,route_id_dir_22_FROM DOWNTOWN,route_id_dir_22_TO DOWNTOWN,route_id_dir_23_FROM DOWNTOWN,route_id_dir_23_TO DOWNTOWN,route_id_dir_24_FROM DOWNTOWN,route_id_dir_24_TO DOWNTOWN,route_id_dir_25_NORTHBOUND,route_id_dir_25_SOUTHBOUND,route_id_dir_28_FROM DOWNTOWN,route_id_dir_28_TO DOWNTOWN,route_id_dir_29_FROM DOWNTOWN,route_id_dir_29_TO DOWNTOWN,route_id_dir_34_FROM DOWNTOWN,route_id_dir_34_TO DOWNTOWN,route_id_dir_35_FROM DOWNTOWN,route_id_dir_35_TO DOWNTOWN,route_id_dir_38_FROM DOWNTOWN,route_id_dir_38_TO DOWNTOWN,route_id_dir_3_FROM DOWNTOWN,route_id_dir_3_TO DOWNTOWN,route_id_dir_41_FROM DOWNTOWN,route_id_dir_41_TO DOWNTOWN,route_id_dir_42_FROM DOWNTOWN,route_id_dir_42_TO DOWNTOWN,route_id_dir_43_FROM DOWNTOWN,route_id_dir_43_TO DOWNTOWN,route_id_dir_4_FROM DOWNTOWN,route_id_dir_4_TO DOWNTOWN,route_id_dir_50_FROM DOWNTOWN,route_id_dir_50_TO DOWNTOWN,route_id_dir_52_FROM DOWNTOWN,route_id_dir_52_TO DOWNTOWN,route_id_dir_55_FROM DOWNTOWN,route_id_dir_55_TO DOWNTOWN,route_id_dir_56_FROM DOWNTOWN,route_id_dir_56_TO DOWNTOWN,route_id_dir_5_FROM DOWNTOWN,route_id_dir_5_TO DOWNTOWN,route_id_dir_64_FROM RIVERFRONT,route_id_dir_64_TO RIVERFRONT,route_id_dir_6_FROM DOWNTOWN,route_id_dir_6_TO DOWNTOWN,route_id_dir_72_EDMONDSON,route_id_dir_72_GRASSMERE,route_id_dir_75_NORTHBOUND,route_id_dir_75_SOUTHBOUND,route_id_dir_76_LOOP,route_id_dir_79_EASTBOUND,route_id_dir_79_NORTHBOUND,route_id_dir_7_FROM DOWNTOWN,route_id_dir_7_TO DOWNTOWN,route_id_dir_84_FROM NASHVILLE,route_id_dir_84_TO NASHVILLE,route_id_dir_86_FROM NASHVILLE,route_id_dir_86_TO NASHVILLE,route_id_dir_8_FROM DOWNTOWN,route_id_dir_8_TO DOWNTOWN,route_id_dir_93_LOOP,route_id_dir_94_FROM NASHVILLE,route_id_dir_95_FROM NASHVILLE,route_id_dir_96_FROM NASHVILLE,route_id_dir_96_TO NASHVILLE,route_id_dir_9_FROM DOWNTOWN,route_id_dir_9_TO DOWNTOWN,is_holiday_False,is_holiday_True
0,0.00,1916.0,53.0,0.0,0.0,0.0,13.0,10.0,0.508382,0.1975,0.0,0.214028,0.04171,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.01,1918.0,53.0,0.0,0.0,0.0,13.0,11.0,0.508382,0.1975,0.0,0.214028,0.04171,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.00,1920.0,53.0,0.0,0.0,0.0,13.0,12.0,0.508382,0.1975,0.0,0.214028,0.04171,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.00,1922.0,53.0,0.0,0.0,0.0,13.0,13.0,0.508382,0.1975,0.0,0.214028,0.04171,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

(5, 96)


,offs
0,0.0


In [26]:
# define model
model = tf.keras.Sequential()
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss='mse',
    metrics=['mae']
)

input_shape = (None, None, len(train_df.columns))
model.build(input_shape)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 256)         361472    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 6

In [27]:
from tensorflow.keras import backend as K
K.clear_session()

In [28]:
checkpoint_filepath = 'models/ons_offs/model/offs_model/CLA_cp-epoch{epoch:02d}-loss{val_loss:.2f}.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [29]:
# fit model
callbacks = [keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True), model_checkpoint_callback]

history = model.fit(dataset_train, validation_data=dataset_val, epochs=5, callbacks=callbacks, verbose=1)

2022-08-18 02:17:49.771957: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5952819456 exceeds 10% of free system memory.


Epoch 1/5
30278/30278 [==============================] - 2052s 68ms/step - loss: 1.3555e-04 - mae: 0.0047 - val_loss: 1.6524e-04 - val_mae: 0.0056
Epoch 2/5
30278/30278 [==============================] - 2647s 87ms/step - loss: 1.2857e-04 - mae: 0.0046 - val_loss: 1.6535e-04 - val_mae: 0.0056
Epoch 3/5
30278/30278 [==============================] - 2726s 90ms/step - loss: 1.0739e-04 - mae: 0.0043 - val_loss: 1.6464e-04 - val_mae: 0.0055
Epoch 4/5
30278/30278 [==============================] - 2719s 90ms/step - loss: 1.1207e-04 - mae: 0.0044 - val_loss: 1.6510e-04 - val_mae: 0.0053
Epoch 5/5
30278/30278 [==============================] - 2750s 91ms/step - loss: 1.0926e-04 - mae: 0.0044 - val_loss: 1.6608e-04 - val_mae: 0.0049
